This notebook reproduces table 1 and figure 4 of the paper : Amortized Tree Generation for Bottom-up Synthesis Planning and Synthesizable Molecular Design.

In [1]:
import rdkit.Chem as Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
import pandas as pd

In [2]:
# Load data resulting from synthesis planning
data = pd.read_csv("../results/demo-inference/decoded_results.csv.gz", compression='gzip')
data.head()

,targets,decoded,similarity
0,COc1cc(Cn2c(C)c(Cc3ccccc3)c3c2CCCC3)ccc1OCC(=O...,COc1cc(Cn2c(C)c(Cc3ccccc3)c3c2CCCC3)ccc1OCC(=O...,1.000000
1,CCC1CCCC(Nc2cc(C(F)(F)F)c(Cl)cc2SC)CC1,NaN,0.000000
2,Clc1cc(Cl)c(C2=NC(c3cccc4c(Br)cccc34)=NN2)nn1,Clc1cc(Cl)c(C2=NC(c3cccc4c(Br)cccc34)=NN2)nn1,1.000000
3,COc1ccc(S(=O)(=O)c2ccc(-c3nc(-c4cc(B(O)O)ccc4O...,COc1ccc(S(=O)(=O)c2ccc(-c3nc(-c4cc(B(O)O)ccc4O...,1.000000
4,CNS(=O)(=O)c1ccc(-c2cc3c4c(ccc3[nH]2)CCCN4C(N)...,CNS(=O)(=O)c1ccc(-c2cc3cc4c(cc3[nH]2)N(C(N)=O)...,0.677419


Inspired from script "23-evaluate-predictions.py"

Clean-up the data, separating recovered (similarity =1) from unrecovered (similarity $\in ]0,1[$) and discarding NaNs (similarity = 0)

In [3]:
import numpy as np
from tdc import Evaluator

"Dataframe with target- and prediction smiles and similarities (*.csv.gz)."
input_file = "../results/demo-inference/decoded_results.csv.gz"


# Keep track of successfully and unsuccessfully recovered molecules
recovered = pd.DataFrame({"targets": [], "decoded": [], "similarity": []})
unrecovered = pd.DataFrame({"targets": [], "decoded": [], "similarity": []})

# load each file containing the predictions
similarity = []
n_recovered = 0
n_unrecovered = 0
n_total = 0
files = [input_file]  # TODO: not sure why the loop but let's keep it for now
for file in files:
    print(f"Evaluating file: {file}")

    result_df = pd.read_csv(file)
    n_total += len(result_df["decoded"])

    # Split smiles, discard NaNs
    is_recovered = result_df["similarity"] == 1.0
    unrecovered = pd.concat([unrecovered, result_df[~is_recovered].dropna()], ignore_index=True)
    recovered = pd.concat([recovered, result_df[is_recovered].dropna()], ignore_index=True)

    n_recovered += len(recovered)
    n_unrecovered += len(unrecovered)
    similarity += unrecovered["similarity"].tolist()

Evaluating file: ../results/demo-inference/decoded_results.csv.gz


In [4]:
unrecovered

,targets,decoded,similarity
0,CNS(=O)(=O)c1ccc(-c2cc3c4c(ccc3[nH]2)CCCN4C(N)...,CNS(=O)(=O)c1ccc(-c2cc3cc4c(cc3[nH]2)N(C(N)=O)...,0.677419
1,CC(NC(=O)C1Cn2c(O)nnc2CN1)c1cc(F)ccc1N1CCC(n2n...,CC(O)c1nnnn1CC1CCN(c2ccc(NC(=O)C3CCc4nnc(-c5cc...,0.286885
2,CCCn1c(C)nnc1CC(C)(O)C(=C(C)C)c1nccnc1S(=O)(=O)F,CCCn1c(C)nnc1CC(O)(c1nccnc1S(=O)(=O)F)c1ncnn1C...,0.590909
3,CN(c1ccccc1)c1ccc(-c2nc3ncccc3s2)cn1,CN(c1ccccc1)c1ccc(C2=NC(c3nsc4cccnc34)=NN2)cn1,0.500000
4,COc1cc(-c2nc(-c3ccc(F)cc3)c(-c3ccc(F)cc3)n2c2c...,COc1cc(-c2nc(-c3ccc(F)cc3)c(-c3ccc(F)cc3)n2c2c...,0.898734


Compute N = number of molecules used, Recovery Rate, Average Similarity, KL Divergence, FC Distance

In [5]:
# Print general info
print(f"N total {n_total}")
recovery_rate = n_recovered/n_total
print(f"Recovery rate {recovery_rate:.2f}%")

n_finished = n_recovered + n_unrecovered
n_unfinished = n_total - n_finished
print(f"N finished tree {n_finished} ({n_finished/n_total:.2f}%)")

average_similarity = np.mean(similarity)
print(f"Average similarity (unrecovered only) {average_similarity}")

temp = []
# Evaluate on TDC evaluators
for metric in "KL_divergence FCD_Distance".split():
    evaluator = Evaluator(name=metric)
    try:
        score_recovered = evaluator(recovered["targets"], recovered["decoded"])
        score_unrecovered = evaluator(unrecovered["targets"], unrecovered["decoded"])
    except TypeError:
        # Some evaluators only take 1 input args, try that.
        score_recovered = evaluator(recovered["decoded"])
        score_unrecovered = evaluator(unrecovered["decoded"])
    except Exception as e:
        logger.error(f"{e.__class__.__name__}: {str(e)}")
        logger.error(e)
        score_recovered, score_unrecovered = np.nan, np.nan

    print(f"Evaluation metric for {evaluator.name}:")
    print(f"    Recovered score: {score_recovered:.2f}")
    temp.append(score_recovered)
    print(f"  Unrecovered score: {score_unrecovered:.2f}")
    temp.append(score_unrecovered)

kl_divergence = temp[0:2]
fc_distance = temp[2:4]

N total 10
Recovery rate 0.40%
N finished tree 9 (0.90%)
Average similarity (unrecovered only) 0.5907895737729258
Evaluation metric for kl_divergence:
    Recovered score: 1.00
  Unrecovered score: 0.59
Evaluation metric for fcd_distance:
    Recovered score: -0.00
  Unrecovered score: 18.28


In [27]:
d = {"N": n_total, "Recovery Rate ": recovery_rate, "Average Similarity ": average_similarity, "KL Divergence ": kl_divergence[0], "FC Distance ": fc_distance[0]}
pd.Series(data=d)

N                      10.000000
Recovery Rate           0.400000
Average Similarity      0.590790
KL Divergence           1.000000
FC Distance            -0.000021
dtype: float64

TODO: reproduce figure 4